In [82]:
import numpy as np
import pandas as pd

## 1. 데이터 로드 & 중심점 없는 학습 지역 보기

In [83]:
df = pd.read_excel('전체_학습_지역_nullable.xlsx')
df.head()

,시도,시군구,행정구역명,법정동,행정구역분류(개정_8자리),법정동코드,중심점_x,중심점_y
0,서울특별시,종로구,청운효자동,청운동,11010720,1111010100,953149.184527,1.954555e+06
1,서울특별시,종로구,청운효자동,신교동,11010720,1111010200,953024.187713,1.954035e+06
2,서울특별시,종로구,청운효자동,궁정동,11010720,1111010300,953444.997666,1.954054e+06
3,서울특별시,종로구,청운효자동,효자동,11010720,1111010400,953377.196082,1.953829e+06
4,서울특별시,종로구,청운효자동,창성동,11010720,1111010500,953421.941975,1.953595e+06


In [84]:
df_null = df[df['중심점_x'].isnull()]
print(len(df_null))
print(df_null.iloc[:5, :])

60
         시도  시군구 행정구역명  법정동  행정구역분류(개정_8자리)       법정동코드  중심점_x  중심점_y
1069  부산광역시  기장군   일광읍  삼성리        21510111  2671025921    NaN    NaN
1070  부산광역시  기장군   일광읍  이천리        21510111  2671025922    NaN    NaN
1071  부산광역시  기장군   일광읍   학리        21510111  2671025923    NaN    NaN
1072  부산광역시  기장군   일광읍  동백리        21510111  2671025924    NaN    NaN
1073  부산광역시  기장군   일광읍  문중리        21510111  2671025925    NaN    NaN


## 2. 중심점 메꾸기

In [85]:
from pyproj import Transformer
import math

In [86]:
import googlemaps
def load_key():
    f = open("api_key.txt", 'r')
    line = f.readline()
    f.close()
    return line
gmap = googlemaps.Client(key=load_key())

In [87]:
def request_location(gmap, location):
    try:
        result_location = gmap.geocode(location)[0]['geometry']['location']
        return result_location['lng'], result_location['lat']
    except: 
        print(f'{location}의 위치를 찾을 수 없습니다.')
        return np.nan, np.nan

In [88]:
# 법정동 전체 주소 만들기
def make_addr(row):
    if row['법정동'][-1] == '리': 
        return f"{row['시도']} {row['시군구']} {row['행정구역명']} {row['법정동']}"
    return f"{row['시도']} {row['시군구']} {row['법정동']}"

In [89]:
coor_transformer = Transformer.from_crs("epsg:4326", "epsg:5179")

for i, row in df.iterrows():
    if math.isnan(row['중심점_x']):
        addr = make_addr(row)
        lng, lat = request_location(gmap, addr)
        y, x = coor_transformer.transform(lat, lng)
        df.loc[i, '중심점_x'] = x
        df.loc[i, '중심점_y'] = y

널 값 확인

In [90]:
df_null = df[df['중심점_x'].isnull()]
print(len(df_null))

0


In [91]:
df.to_csv('town.csv', index=False)